## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-10-14-53-56 +0000,nyt,R.F.K. Allies Embrace ‘Anti-Vax’ Label They On...,https://www.nytimes.com/2025/11/10/us/politics...
1,2025-11-10-14-53-05 +0000,nyt,"Nicolas Sarkozy, Former French President, Will...",https://www.nytimes.com/2025/11/10/world/europ...
2,2025-11-10-14-51-19 +0000,nyt,BBC Director Tim Davie and CEO Deborah Turness...,https://www.nytimes.com/2025/11/09/world/europ...
3,2025-11-10-14-50-32 +0000,nypost,Witnesses describe horrifying sound after Disn...,https://nypost.com/2025/11/10/us-news/witnesse...
4,2025-11-10-14-48-00 +0000,wsj,Supreme Court Declines to Reconsider Same-Sex ...,https://www.wsj.com/us-news/law/supreme-court-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2327/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
26,trump,39
57,shutdown,29
56,government,13
97,deal,12
85,end,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
238,2025-11-09-18-46-21 +0000,nypost,SNAP thrown into chaos after Trump admin tells...,https://nypost.com/2025/11/09/us-news/snap-thr...,99
158,2025-11-10-05-02-42 +0000,nyt,Trump Renews Attacks on Obamacare in New Push ...,https://www.nytimes.com/2025/11/09/us/politics...,99
20,2025-11-10-14-25-00 +0000,wsj,The decision by eight \members of the Senate D...,https://www.wsj.com/politics/policy/democrats-...,91
200,2025-11-09-22-28-52 +0000,nypost,Senate eyes vote on potential breakthrough to ...,https://nypost.com/2025/11/09/us-news/senate-e...,87
44,2025-11-10-13-44-58 +0000,nypost,"Democrats melt down, call for Chuck Schumer’s ...",https://nypost.com/2025/11/10/us-news/democrat...,86


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
238,99,2025-11-09-18-46-21 +0000,nypost,SNAP thrown into chaos after Trump admin tells...,https://nypost.com/2025/11/09/us-news/snap-thr...
20,62,2025-11-10-14-25-00 +0000,wsj,The decision by eight \members of the Senate D...,https://www.wsj.com/politics/policy/democrats-...
38,45,2025-11-10-14-01-26 +0000,nypost,Ex-French President Nicolas Sarkozy to be rele...,https://nypost.com/2025/11/10/world-news/franc...
93,39,2025-11-10-11-00-00 +0000,latimes,California Supreme Court rejects free-speech c...,https://www.latimes.com/california/story/2025-...
111,38,2025-11-10-09-54-20 +0000,cbc,"Typhoon Fung-wong blows away from Philippines,...",https://www.cbc.ca/news/world/typhoon-fung-won...
189,34,2025-11-10-00-12-31 +0000,nypost,Boca Raton makes serious pitch to NYC business...,https://nypost.com/2025/11/09/us-news/boca-rat...
191,32,2025-11-10-00-09-50 +0000,nypost,Brooklyn dad says goodbye to hero son Itay Che...,https://nypost.com/2025/11/09/world-news/nyc-d...
64,31,2025-11-10-12-30-00 +0000,wsj,Private-Label Food Maker TreeHouse to Go Priva...,https://www.wsj.com/business/retail/private-la...
2,29,2025-11-10-14-51-19 +0000,nyt,BBC Director Tim Davie and CEO Deborah Turness...,https://www.nytimes.com/2025/11/09/world/europ...
49,29,2025-11-10-13-15-00 +0000,wsj,President Trump pardoned a list of top allies ...,https://www.wsj.com/politics/policy/trump-pard...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
